In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:95% !important; }</style>"))

### Import necessary packages

In [79]:
#Essentials
import numpy as np
import pandas as pd
import pickle
import re
import datetime as dt
import time

# For scraping
import requests
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.support.ui import Select
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
import os

#SQL related - NEED TO DECIDE WHICH ONE I'LL BE USING AND DELETE THE REST
import psycopg2
import sqlite3
from sqlalchemy import create_engine
import pandas.io.sql as pd_sql

#Visualization
import matplotlib.pyplot as plt
import matplotlib
import seaborn as sns


%matplotlib inline
from matplotlib.pylab import rcParams
rcParams['figure.figsize'] = 25, 6
rcParams.update({'font.size': 14})

## Gather data

### Helper functions

In [85]:
def is_for_sale_active(url):
    """Checks if 'For Sale' tab is selected, to make sure we're not looking at rentals
    Returns a Boolean (True for For Sale is active, False - if it isn't active)"""
    
def get_todays_ads():
    """Retrieves only adverts posted today"""
    
#Get urls for all the pages and put them into a list
def get_url_list():
    """Records the current url and goes through the website, clicking Next as many times as there are pages.
    Returns a list of urls to be used in the get_htmls function."""
    #Creates a list of urls for all pages
    url_list = []
    
    #Reads the url of the page the driver is currently in and adds it into the list
    current_page_url = driver.current_url
    url_list.append(current_page_url)
    
    curr_page_num = 0
    count_exceptions = 0
    while True:
        try:
            curr_page_num += 1
            #Find "Next" button and click it
            find_next = driver.find_element_by_xpath("//a[@rel='next']")
            find_next.click()
            current_page_url = driver.current_url
            url_list.append(current_page_url)
            time.sleep(10)
        except:
            #Count the exceptions
            count_exceptions += 1
            curr_page_num -= 1 #since we are coming back to the same page
            #If the there haven't been 3 exceptions yet, sleep for a bit and then continue
            if count_exceptions < 3:
                time.sleep(10)
            else:
                #If "Next" button isn't there anymore or an error occurs, return the list
                #driver.close()
                print(f'Last page number was {curr_page_num}.')
                return url_list
    print(f'Last page number was {curr_page_num}.')
    #driver.close()
    return url_list

#Get all the html files for each property ad and put it into a list
def get_htmls(url_list):
    """Takes an url (of one page) and an existing html list.
    Returns updated html list with all the htmls from provided url."""
    
    
    #Find all announcement items
    elems = driver.find_elements_by_xpath("//a[@href]")
    for elem in elems:
        # NEED TO STRIP OUT NON-PROPERTY AD HREFs
        html = elem.get_attribute("href")
        html_list.append(html)
        
        print(elem.get_attribute("href"))
        
    return html_list

# IT MIGHT BE EASIER TO GO THROUGH URLs AND GET THE hrefs PAGE BY PAGE

In [75]:
# At the moment only getting the data from Funda, could have a list of websites in the future
website = 'https://www.funda.nl/en/'

chromedriver = "/Applications/chromedriver"           #  path to the chromedriver executable
os.environ["webdriver.chrome.driver"] = chromedriver

In [82]:
# Go to the website
driver = webdriver.Chrome(chromedriver)
driver.get(website)

In [83]:


# Now apply desired filters (e.g. set location to Amsterdam)
# try:
filter_loc = driver.find_element_by_name('filter_location')
filter_loc.send_keys('Amsterdam')

# Wait for the first dropdown option to appear and select the first option
WebDriverWait(driver, 5).until(EC.visibility_of_element_located((By.CSS_SELECTOR, ".autocomplete-list")))
filter_loc.send_keys(Keys.ARROW_DOWN)
filter_loc.send_keys(Keys.ENTER)

filter_radius = Select(driver.find_element_by_id('Straal'))
filter_radius.select_by_value('0')

filter_price_from = Select(driver.find_element_by_name('filter_KoopprijsVan'))
filter_price_from.select_by_value('200000')

filter_price_upto = Select(driver.find_element_by_name('filter_KoopprijsTot'))
filter_price_upto.select_by_value('400000')

find_search = driver.find_element_by_xpath("//button[contains(text(), 'Search')]")
find_search.click()

# Now that we applied basic filters, let's narrow the search down more

# except:
#     driver.find_element_by_class_name('recaptcha-checkbox-border').click()
# except:
#     WebElement iFrame_checkbox = driver.findElement(By.xpath("xpath_of_reCaptcha_checkbox"));
#     iFrame_checkbox.click();


# car_type_form.send_keys(Keys.RETURN)

In [91]:
# url_list = get_url_list()
# with open('./Cellar/pages_url_list.pkl', 'wb') as url_pickle:
#     pickle.dump(url_list, url_pickle)

In [96]:
with open('./Cellar/pages_url_list.pkl', 'rb') as url_pickle:
    url_list = pickle.load(url_pickle)

print(f'Imported {len(url_list)} URLs.')

Imported 77 URLs.


In [101]:
elems = driver.find_elements_by_xpath("//a[@href]")
for elem in elems:
    print(elem.get_attribute("href"))

https://www.funda.nl/en/
https://www.funda.nl/en/koop/amsterdam/200000-400000/p77/cettzercavddqudtb.html
https://www.funda.nl/en/navigation/menu/
https://www.funda.nl/en/huis-plaatsen/
https://www.funda.nl/en/makelaars/
https://www.funda.nl/en/makelaarsmatch/
https://www.funda.nl/thuis/verkopen/
https://www.funda.nl/en/woningwaarde/
https://www.funda.nl/en/hypotheek/maandlasten/de-hypotheker/
https://www.funda.nl/thuis/kopen/
https://www.funda.nl/en/navigation/account/
https://www.funda.nl/en/mijn/login/?ReturnUrl=%2Fen%2Fkoop%2Famsterdam%2F200000-400000%2F
https://www.funda.nl/en/mijn/aanmelden/?ReturnUrl=%2Fen%2Fkoop%2Famsterdam%2F200000-400000%2F
https://www.funda.nl/en/koop/amsterdam/200000-400000/p77/cettzercavddqudtb.html
https://www.funda.nl/en/navigation/language/
https://www.funda.nl/language/switch/?returnUrl=%2Fen%2Fkoop%2Famsterdam%2F200000-400000%2Fp77%2F
https://www.funda.nl/en/mijn/nieuw/?Context=System.Web.HttpContextWrapper&Request=System.Web.HttpRequestWrapper&Respons

In [81]:
driver.close()